In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/Preprocesing/Datos/train_stereohoax_df_a3_preprocesado.csv')
dfa = pd.read_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/Dividido/train_stereohoax_df.csv')

# Conservar solo las columnas 'id' y 'text'
df = df[['id', 'text']]
dfa = dfa[['id','label']]

# Unir los DataFrames en base a la columna 'id'
df_final = pd.merge(df, dfa, on='id', how='inner')

# Guardar el DataFrame resultante en un nuevo archivo CSV
df_final.to_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/train_detests_preprocesado_a_backtranslation.csv', index=False)


In [ ]:
import pandas as pd

# Cargar el archivo CSV en un DataFrame
df2 = pd.read_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/Preprocesing/Datos/valid_stereohoax_df_a3_preprocesado.csv')
df2a = pd.read_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/Dividido/valid_stereohoax_df.csv')

# Conservar solo las columnas 'id' y 'text'
df2 = df2[['id', 'text']]
df2a = df2a[['id','label']]

# Unir los DataFrames en base a la columna 'id'
df_final2 = pd.merge(df, dfa, on='id', how='inner')

# Guardar el DataFrame resultante en un nuevo archivo CSV
df_final2.to_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/valid_detests_preprocesado_a_backtranslation.csv', index=False)


# Configuración (importar dependencias, librerías, ...)

In [ ]:
# Set the seed value all over the place to make this reproducible.
# esto hay que ponerlo justo antes de importar para que los experimentos
# sean reproducible

!pip install pytorch-lightning
import random
import torch
import numpy as np
import os
from pytorch_lightning import seed_everything

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)# Store the average loss after eachepoch so we can plot them.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
os.environ["TF_DETERMINISTIC_OPS"] = "1" # See:https://github.com/NVIDIA/tensorflow-determinism#confirmed-current-gpu-specific-sources-of-non-determinism-with-solutions
seed_everything(42, workers=True)

!pip install transformers datasets
# # !pip install --upgrade accelerate
!pip install sentencepiece
!pip install contractions
!pip install textblob
from datasets import Dataset, DatasetDict, load_metric
import pandas as pd
import sklearn as sk
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, average_precision_score, f1_score
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
 TrainingArguments, Trainer, pipeline, EarlyStoppingCallback

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 17.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-man

INFO:lightning_fabric.utilities.seed:Seed set to 42


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
# Check that pyTorch is identifying the GPU
if torch.cuda.device_count() > 0:
    # If a GPU is available, print its name
    print(f'GPU detected. Currently using: "{torch.cuda.get_device_name(0)}"')
    # Set the device to GPU for accelerated computations
    device = torch.device("cuda")
else:
    # If no GPU is available, inform the user to change the runtime type
    print('Currently using CPU. To utilize GPU acceleration, change the runtime type in the \'runtime\' tab.')

GPU detected. Currently using: "Tesla T4"


# Preparación de los datos

## Lectura de los ficheros

In [ ]:
# Usamos estas variables para que el código sea más portable
nombre_etiqueta = 'label'
campo_texto = 'text'

### Ficheros divididos

In [ ]:
# Leemos los ficheros de datos divididos
data_path = '/content/drive/MyDrive/TFG/Task 1/Stereohoax/'
train_df = pd.read_csv(data_path + 'train_detests_preprocesado_a_backtranslation.csv')
valid_df = pd.read_csv(data_path + 'valid_stereohoax_preprocesado_a_backtranslation.csv')

In [ ]:
# Para saber el número de filas de cada clase en cada división
print("Distribución original - train: ",train_df.value_counts(nombre_etiqueta))
print("Distribución original - Valid: ",valid_df.value_counts(nombre_etiqueta))

Distribución original - train:  label
0    2121
1     872
Name: count, dtype: int64
Distribución original - Valid:  label
0    607
1    249
Name: count, dtype: int64


In [ ]:
train_df.head(5)

In [ ]:
valid_df.head(5)

# Backtranslation (Data augmentation)

In [ ]:
# Función para realizar la traducción
def backtranslate(text, model1, tokenizer1, model2, tokenizer2, target_lang='es'):
    print(text)
    # Traducción a inglés
    translated_text = model1.generate(**tokenizer1(text, return_tensors='pt', padding=True, truncation=True, max_length=512), max_length=512).to('cpu').numpy()
    translated_text = tokenizer1.decode(translated_text[0], skip_special_tokens=True)
    # print(translated_text)
    # Traducción a español
    backtranslated_text = model2.generate(**tokenizer2(translated_text, return_tensors='pt', padding=True, truncation=True, max_length=512), max_length=512).to('cpu').numpy()
    backtranslated_text = tokenizer2.decode(backtranslated_text[0], skip_special_tokens=True)
    # print(backtranslated_text)
    return backtranslated_text

In [ ]:
# Función para aplicar backtranslation a las instancias
def apply_backtranslation(row, model1, tokenizer1, model2, tokenizer2):
    print(f"ID Doccano: {row['id']}")
    backtranslated_text = backtranslate(row['text'], model1, tokenizer1, model2, tokenizer2)
    # Crear una nueva fila con el mismo id_doccano, id_tweet, anotador1, anotador2 y label
    new_row = row.copy()
    new_row[campo_texto] = backtranslated_text
    return new_row

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
# Iniciar modelos y tokenizadores
model_checkpoint_trans = 'Helsinki-NLP/opus-mt-es-en'
model_trans = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_trans)
tokenizer_trans = AutoTokenizer.from_pretrained(model_checkpoint_trans)
model_checkpoint_back = 'Helsinki-NLP/opus-mt-en-es'
model_back = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint_back)
tokenizer_back = AutoTokenizer.from_pretrained(model_checkpoint_back)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
# Aplicar backtranslation a train
new_rows_train = []
for _, row in train_df.iterrows():
    # Solo a las instancias con la eiqueta 'SI'
    if row[nombre_etiqueta] == 1:
        new_rows_train.append(apply_backtranslation(row, model_trans, tokenizer_trans, model_back, tokenizer_back))
# Concatenar train con las nuevas instancias
train_df_back = pd.concat([train_df, pd.DataFrame(new_rows_train)], ignore_index = True)
# Ordenar de manera aleatoria
train_df_back = train_df_back.sample(frac = 1, random_state = 42).reset_index(drop = True)

ID Doccano: s_4073
falso. se ha demostrado que el chico los atacó sin razón (sin más razón que su flagrante racismo) y los chicos lo redujeron. en el video (el primero) se ve cómo solo lo están inmovilizando.
¿siempre tenéis que mentir para contestar cualquier cosa?
ID Doccano: s_1471
vete a tu país imbécil.
ID Doccano: s_821
a ver si lo he entendido bien. según tu, puesto que no aportan nada se les tiene que denegar salud. tu y los derechos humanos como que no eh? soy inmigrante legal. y aún así tuvimos muchos problemas, tanto burocráticos como de sociedad. y esto señor mío, es racismo y xenofobia.
ID Doccano: s_869
en la sanidad pública pagas por los extranjeros que vienen a operarse por la cara, y no hablo sólo de inmigrantes ilegales, es que le estás pagando tratamientos a gente que viene aquí a operarse gratis o por medicamentos que en sus países son carísimos. aquí se lo pagas tú.
ID Doccano: s_328
una agenda que consiste básicamente en subir los impuestos para pagar todas las ay

In [ ]:
# Aplicar backtranslation a valid
new_rows_valid = []
for _, row in valid_df.iterrows():
    # Solo a las instancias con la eiqueta 'SI'
    if row[nombre_etiqueta] == 1:
        new_rows_valid.append(apply_backtranslation(row, model_trans, tokenizer_trans, model_back, tokenizer_back))
# Concatenar valid con las nuevas instancias
valid_df_back = pd.concat([valid_df, pd.DataFrame(new_rows_valid)], ignore_index = True)
# Ordenar de manera aleatoria
valid_df_back = valid_df_back.sample(frac = 1, random_state = 42).reset_index(drop = True)

ID Doccano: s_1496
A este sectario no se le puede echar de España??
ID Doccano: s_1315
Con suerte sabe andar y respirar a la vez sin cagarse encima.
@user @user
ID Doccano: s_1529
Pues si no le gusta, de vuelta a su puta casa.
ID Doccano: s_1383
Por tierra, mar o aire.
Como quieras, lárgate, Serigne.
Nuestras tradiciones se quedan.
Y más las de los niños. SON SAGRADOS.
Sóis escoria.
ID Doccano: s_4041
5.23 INMIGRACIÓN
Uno de "ntros niños" intentó abusar de una chica y al verse rodeado por la Policía Local de Bilbao,esgrimió un cuchillo y se tiró a la ría.
¿Hasta cuándo seguiremos importando chusma y premiándola con una paga?
Deportacion
(@user)
URL
ID Doccano: s_3686
El alcalde de Pamplona, Enrique Maya, ha asegurado este martes que "casi todas las personas que cometen delitos no son nacidas aquí"  URL
ID Doccano: s_1641
Si no le gustan las tradiciones de España que se vaya a su país a vivir las suyas.
Siempre estamos igual;
En mi país todo es más bonito, en mi país la gente es más edu

In [ ]:
# Para saber el número de filas de cada clase en cada división
print("Distribución original - train Backtranslated: ",train_df_back.value_counts(nombre_etiqueta))
print("Distribución original - Valid Backtranslated: ",valid_df_back.value_counts(nombre_etiqueta))

Distribución original - train Backtranslated:  label
0    2121
1    1744
Name: count, dtype: int64
Distribución original - Valid Backtranslated:  label
0    607
1    498
Name: count, dtype: int64


In [ ]:
train_df_back.head(15)

In [ ]:
valid_df_back.head(15)

In [ ]:
train_df_back.to_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/BackTranslation/Datos/train_stereohoax_bt.csv', index = False)
valid_df_back.to_csv('/content/drive/MyDrive/TFG/Task 1/Stereohoax/BackTranslation/Datos/valid_stereohoax_bt.csv', index = False)